In [8]:
import json
import random
from bioc import biocjson
import pandas as pd

# Load papers discussing mutations

In [9]:
# Load papers discussing SARS-CoV-2 variants
with open('data_bioc.txt') as file:
        data_bioc = json.loads(file.read())

In [10]:
data_bioc = {k: v for k, v in data_bioc.items() if v is not None}

# Obtain DOI's
data_doi = []
for x in data_bioc.keys():
     data_bioc.append(x.split("doi.org/")[-1])

# Load all filtered papers

In [11]:
# Load filtered_papers
with open('filtered_papers.txt') as file:
        filtered_papers = json.loads(file.read())

In [12]:
def related_paper(paper):
    try:
        doi = paper["passages"][0]['infons']['article-id_doi']
        
        if doi in data_doi:
            return True
            
    except:
        return False

    return False

In [13]:
filtered_papers_copy = [x for x in filtered_papers if not related_paper(x)]

# BioC from API

In [14]:
# Load filtered_papers
with open('publication_bioc.txt') as file:
        publication_bioc= json.loads(file.read())

publication_bioc = {k: v for k, v in publication_bioc.items() if v is not None}

In [15]:
def extract_nested_elements(input_string):
    elements = []
    start = 0
    brace_count = 0
    inside_element = False

    for i, char in enumerate(input_string):
        if char == '{':
            if brace_count == 0:
                start = i
                inside_element = True
            brace_count += 1
        elif char == '}':
            brace_count -= 1
            if brace_count == 0 and inside_element:
                elements.append(input_string[start:i+1])
                inside_element = False

    return elements

# Example usage
# input_string = "{apple, {fruit: {type: citrus, examples: {lemon, orange}}}}, {banana}, {cherry, {type: berry}}, {date}"
# custom_list = extract_nested_elements(input_string)
# print(custom_list)

In [16]:
# l = extract_nested_elements(publication_bioc["https://doi.org/10.1126/science.abl6184"])

In [17]:
# Extract text from data_bioc
publication_text = []
count = 0 

for paper in publication_bioc.values():
    text = ""
    
    # count += 1
    # if count > 2:
    #     break

    if count == 1:
        break
    # print(paper)
    paper = paper[1:-1]
    
    try:
        bioc_list = extract_nested_elements(paper)
        # for bioc in bioc_list:
        
        bioc_collection = biocjson.loads(bioc_list[-1])
        # bioc_collection = biocjson.loads(paper)
        
    except:
        count += 1
        # print(paper)
        print(paper)
        print("")
        print(paper[42506:])
        # print(paper[42505])
        # print(count)
        continue
    
    for document in bioc_collection.documents:    
        for passage in document.passages:
            try:
                text += passage.text
            except:
                print(passage)
                pass
   
    if text == "":
        count += 1
        continue

    publication_text.append(text)

In [18]:
len(publication_text)

195

In [ ]:
count

In [ ]:
# publication_bioc["https://doi.org/10.1126/science.abl6184"][42506:]

In [ ]:
# publication_bioc["https://doi.org/10-1093_cid_ciaa1474"]

# BioC from pdf

In [ ]:
# Extract text from unlabelled
unlabelled_text = []

for paper in unlabelled:
    
    try:
        passage = paper["passages"]
    except:
        continue
    
    # authors = paper["authors"]
    # pmid = paper["id"]
    # year = paper["year"]
    text = ""

    for section in passage:
        # print(" ")
        # print(section) 
        try:
            text += section['text']
        except:
            pass

    unlabelled_text.append(text)

In [ ]:
# Extract text from data_bioc
data_text = []
count = 0 

for paper in data_bioc.values():
    text = ""
    
    # count += 1
    # if count > 2:
    #     break

    # if count == 1:
    #     break
    # print(paper)
    paper = paper[1:-1]
    
    # bioc_list = extract_nested_elements(paper)
        
    # bioc_collection = biocjson.loads(bioc_list[-1])
    
    try:
        # Some BioC repeat
        bioc_list = extract_nested_elements(paper)
        # for bioc in bioc_list:
        
        bioc_collection = biocjson.loads(bioc_list[-1])
        
    except:
        count += 1
        # print(paper)
        print(bioc_list[-1])
        
        print(len(bioc_list))
        # print(paper[42506:])
        # print(paper[42505])
        # print(count)
        continue
    
    for document in bioc_collection.documents:    
        for passage in document.passages:
            try:
                text += passage.text
            except:
                print(passage)
                pass
   
    if text == "":
        count += 1
        continue

    data_text.append(text)

In [ ]:
count

In [ ]:
len(data_text)

# Create labelled training dataset for binary classification

In [ ]:
unlabelled = random.sample(filtered_papers_copy, len(publication_text))

In [ ]:
unlabelled_string = []
for paper in unlabelled:
    unlabelled_string.append(str(paper))

In [ ]:
len(unlabelled)

In [ ]:
df = pd.DataFrame(unlabelled_string, columns=["text"])
df["label"] = 0

In [ ]:
df_2 = pd.DataFrame(publication_text, columns=["text"])
df_2["label"] = 1

In [ ]:
df = pd.concat([df, df_2])

In [ ]:
df.to_csv("bn_pub_dataset.csv")